In [64]:
import geopandas as gpd
import maup

precincts = gpd.read_file("./shapefiles/fl/fl_2016.shp")
units = gpd.read_file("./shapefiles/fl/tabblock2010_12_pophu.shp")


0       POLYGON Z ((943790.734 591023.581 0.000, 94364...
1       POLYGON Z ((942137.421 562243.019 0.000, 94093...
2       POLYGON Z ((873353.097 533771.394 0.000, 87420...
3       POLYGON Z ((936263.331 593379.375 0.000, 93560...
4       POLYGON Z ((940693.858 597222.143 0.000, 94069...
                              ...                        
5962    POLYGON Z ((-866918.391 2432253.809 0.000, -86...
5963    POLYGON Z ((-877881.043 2406385.282 0.000, -87...
5964    POLYGON Z ((-824547.862 2411437.171 0.000, -82...
5965    POLYGON Z ((-848746.355 2379671.775 0.000, -84...
5966    POLYGON Z ((-816442.346 2361650.824 0.000, -81...
Name: geometry, Length: 5967, dtype: geometry

In [65]:
precincts["geometry"] = maup.make_valid(blocks)

In [66]:
units["geometry"] = maup.make_valid(precincts)

In [67]:
units.to_crs(precincts.crs, inplace=True)

assignment = maup.assign(blocks, precincts)

/Users/sherzodnimatullo/code/projects/416/gerrychain/env/lib/python3.10/site-packages/maup/indexed_geometries.py:17: ShapelyDeprecationWarning: STRtree will be changed in 2.0.0 and will not be compatible with versions < 2.
  self.spatial_index = STRtree(self.geometries)
/Users/sherzodnimatullo/code/projects/416/gerrychain/env/lib/python3.10/site-packages/maup/indexed_geometries.py:17: ShapelyDeprecationWarning: STRtree will be changed in 2.0.0 and will not be compatible with versions < 2.
  self.spatial_index = STRtree(self.geometries)


In [68]:
precincts["POP10"] = units["POP10"].groupby(assignment).sum()
precincts["C_X"] = precincts.centroid.x
precincts["C_Y"] = precincts.centroid.y

from gerrychain import Graph

dual_graph = Graph.from_geodataframe(precincts)
dual_graph.add_data(precincts)
dual_graph.to_json("./FL_VTD.json")

/Users/sherzodnimatullo/code/projects/416/gerrychain/env/lib/python3.10/site-packages/gerrychain/graph/adjacency.py:21: ShapelyDeprecationWarning: Setting custom attributes on geometry objects is deprecated, and will raise an AttributeError in Shapely 2.0
  geometries[i].id = i
/Users/sherzodnimatullo/code/projects/416/gerrychain/env/lib/python3.10/site-packages/gerrychain/graph/adjacency.py:23: ShapelyDeprecationWarning: STRtree will be changed in 2.0.0 and will not be compatible with versions < 2.
  tree = STRtree(geometries)
/Users/sherzodnimatullo/code/projects/416/gerrychain/env/lib/python3.10/site-packages/gerrychain/graph/adjacency.py:66: UserWarning: Found overlaps among the given polygons. Indices of overlaps: {(3666, 4415), (572, 1107), (2467, 2474), (3806, 4004), (194, 1994), (1428, 2288), (1954, 2465), (3286, 4688), (252, 952), (3873, 4521), (1248, 2290), (3592, 3593), (501, 2593), (510, 1232), (761, 1805), (2010, 2353), (819, 2195), (2035, 2382), (5227, 5896), (2864, 4415)

In [71]:
import networkx as nx
import matplotlib.pyplot as plt

plt.figure()
nx.draw(dual_graph, pos={n:(dual_graph.nodes[n]['C_X'],dual_graph.nodes[n]['C_Y']) for n in dual_graph.nodes()}, cmap = 'coolwarm', node_size=30)
plt.savefig("./FL_Dual_Graph.png")


/Users/sherzodnimatullo/code/projects/416/gerrychain/env/lib/python3.10/site-packages/networkx/drawing/nx_pylab.py:433: UserWarning: No data for colormapping provided via 'c'. Parameters 'cmap' will be ignored
  node_collection = ax.scatter(
